In [ ]:
import torch
import torchvision
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Need this for cross-compatibility as we trained and tested on Google Colab GPU
device = torch.device("cuda" if torch.cuda.is_available else "cpu")

# Preprocessing (finding mean and std to normalise) and loading the dataset
temp_transform = transforms.ToTensor()
temp_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=temp_transform)
temp_loader = DataLoader(temp_trainset, batch_size=128, shuffle=False, num_workers=2)

mean = torch.zeros(3)
std = torch.zeros(3)
no_of_batches = 0

for x, y in temp_loader:
    mean += x.mean(dim=[0,2,3])
    std += x.std(dim=[0,2,3])
    no_of_batches += 1

mean /= no_of_batches
std /= no_of_batches
print("Mean: ", mean)
print("Std: ", std)


Files already downloaded and verified


/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <F33ACD4D-1C10-3F4E-ADEC-717A02BDBB52> /opt/anaconda3/envs/myenv/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torchvision/image.so, 

Mean:  tensor([0.4914, 0.4822, 0.4465])
Std:  tensor([0.2467, 0.2432, 0.2612])


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean.tolist(), std.tolist())
])
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

training_set_size = 45000
validation_set_size = 5000

training_dataset, validation_dataset = random_split(cifar_trainset, [training_set_size, validation_set_size])

# Num_workers depends on system cores
train_loader = DataLoader(training_dataset, batch_size=128, shuffle=True, num_workers=2)
validation_loader = DataLoader(validation_dataset, batch_size=128, shuffle=False, num_workers=2)
test_loader = DataLoader(cifar_testset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Starting dimensions 32 x 32 x 64 -> 16 x 16 x 128
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv1_3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bnorm1_1 = nn.BatchNorm2d(64)
        self.bnorm1_2 = nn.BatchNorm2d(64)
        self.bnorm1_3 = nn.BatchNorm2d(64)
        
        # Starting dimensions 16 x 16 x 128 -> 8 x 8 x 128
        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3 ,stride=1, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv2_3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bnorm2_1 = nn.BatchNorm2d(128)
        self.bnorm2_2 = nn.BatchNorm2d(128)
        self.bnorm2_3 = nn.BatchNorm2d(128)


        # Starting dimensions 8 x 8 x 256 -> 4 x 4 x 256
        self.conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bnorm3_1 = nn.BatchNorm2d(256)
        self.bnorm3_2 = nn.BatchNorm2d(256)
        self.bnorm3_3 = nn.BatchNorm2d(256)

        # Reduces H X W dimensions by half (32 x 32 -> 16 x 16)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Activation function
        self.relu = nn.ReLU()

        # Final fully connected layer, input (4 x 4 x 256) dimensions should be flattened
        self.fully_connected_layer = nn.Linear(4*4*256, 10)
        # Cross entropy loss
        self.cross_entropy_loss = nn.CrossEntropyLoss()


    def forward(self, input):
        input = self.relu(self.bnorm1_1(self.conv1_1(input)))
        input = self.relu(self.bnorm1_2(self.conv1_2(input)))
        input = self.relu(self.bnorm1_3(self.conv1_3(input)))
        input = self.maxpool(input)

        input = self.relu(self.bnorm2_1(self.conv2_1(input)))
        input = self.relu(self.bnorm2_2(self.conv2_2(input)))
        input = self.relu(self.bnorm2_3(self.conv2_3(input)))
        input = self.maxpool(input)

        input = self.relu(self.bnorm3_1(self.conv3_1(input)))
        input = self.relu(self.bnorm3_2(self.conv3_2(input)))
        input = self.relu(self.bnorm3_3(self.conv3_3(input)))
        input = self.maxpool(input)

        input = self.fully_connected_layer(torch.flatten(input, 1, -1))

        return input

In [ ]:
# Training

import torch.optim as optim
cnn_model = CNN().to(device)
optimiser = optim.Adam(cnn_model.parameters(), lr=0.001)
accuracy_history = []
loss_history = []
validation_loss_history = []
validation_accuracy_history = []
epochs = 100
for i in range(epochs):
    cnn_model.train()
    total_loss = 0
    training_correct = 0
    for x_train, y_train in train_loader:
        x_train, y_train = x_train.to(device), y_train.to(device)
        optimiser.zero_grad()
        output = cnn_model(x_train)
        loss = cnn_model.cross_entropy_loss(output, y_train)
        loss.backward()
        optimiser.step()
        
        total_loss += loss.item()
        training_correct += (torch.argmax(output, dim=1) == y_train).sum().item()
        
    
    avg_loss = total_loss / len(train_loader)
    training_accuracy = training_correct / len(train_loader.dataset)
    loss_history.append(avg_loss)
    accuracy_history.append(training_accuracy)

    # Validation
    cnn_model.eval()
    validation_loss = 0
    validation_correct = 0

    with torch.no_grad():
        for x_validate, y_validate in validation_loader:
            x_validate, y_validate = x_validate.to(device), y_validate.to(device)
            output = cnn_model(x_validate)
            
            validation_loss += cnn_model.cross_entropy_loss(output, y_validate).item()
            validation_correct += (torch.argmax(output, dim=1) == y_validate).sum().item()
    
    avg_validation_loss = validation_loss / len(validation_loader)
    validation_accuracy = validation_correct / len(validation_loader.dataset)

    validation_loss_history.append(avg_validation_loss)
    validation_accuracy_history.append(validation_accuracy)
    print(f"Epoch {i} Loss : {avg_loss:.4f}, Validation Set Loss: {avg_validation_loss:.4f}, Validation Set Accuracy: {validation_accuracy:.4f}")

In [ ]:
# Testing
total_correct = 0
total_samples = 0
test_loss = 0
cnn_model.eval()
with torch.no_grad():
    for x_test, y_test in test_loader:
        x_test, y_test = x_test.to(device), y_test.to(device)
        output = cnn_model(x_test)
        loss = cnn_model.cross_entropy_loss(output, y_test)
        test_loss += loss.item()
        y_pred = torch.argmax(output, dim=1)
        correct_pred = torch.sum(y_pred == y_test)
        batch_accuracy = correct_pred.item() / len(x_test)
        total_correct += correct_pred.item()
        total_samples += len(x_test)
accuracy = total_correct / total_samples
avg_test_loss = test_loss / len(test_loader)
print("Correct predictions: ", total_correct)
print(f"Accuracy: {accuracy:.2f}, Test Loss: {avg_test_loss:.4f}")